**Load the excel file**

In [1]:
import pandas as pd

path_excel = r'/home/xian/Documents/SUITE_Trans/PostProcessing_JPN_CHN.xlsx'
data = pd.read_excel (path_excel, header=0)
df = pd.DataFrame(data)

#print(repr(df.columns[1])) # show header column name
#print(repr(df.iloc[53,0])) # access a field
source_en = df['English (US)\r\n[ S; en-US ]'].to_list()

**Save extracted data to file**

In [2]:
with open('/home/xian/Documents/SUITE_Trans/output.txt', 'w') as f:
    for item in source_en:
        item = repr(item)
        item = item[1:-1]
        f.write("%s\n" % item)

**Import Google Translate API**

In [4]:
from google.cloud import translate

def translate_text(text="YOUR_TEXT_TO_TRANSLATE", project_id="YOUR_PROJECT_ID"):
    """Translating Text."""

    client = translate.TranslationServiceClient()

    parent = client.location_path(project_id, "global")

    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        parent=parent,
        contents=[text],
        mime_type="text/plain",  # mime types: text/plain, text/html
        source_language_code="en-US",
        target_language_code="fr",
    )
    # Display the translation for each input text provided
    for translation in response.translations:
        print(u"Translated text: {}".format(translation.translated_text))

my_project_id = 'carbide-trees-283313'
translate_text(text=item, project_id=my_project_id)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started